In [ ]:
from my_weapon import *
from collections import defaultdict

In [2]:
url_tweets = json.load(open("data/url_tweets_v2.json"))
# del url_tweets[0] # 第一条too general 目前的数据已经删除

In [3]:
starters_cnt = 0
IRAs_cnt = 0
first_users = set()
starter_users = set()
IRAs = set()

for fake in url_tweets:
    for j, t in enumerate(fake["tweets"]):
        if t["IRA"]:
            IRAs.add(t["user_id"])
            IRAs_cnt += 1
            
        if t["origin"]:
            starters_cnt += 1
            starter_users.add(t["user_id"])
            if j == 0:
                first_users.add(t["user_id"])
    
len(url_tweets), starters_cnt, len(first_users), len(starter_users), len(IRAs), IRAs_cnt

(189620, 914272, 25212, 88215, 109, 912)

In [4]:
from SQLite_handler import find_user_info

starter_users_info = []

with open("data/sources_users_info.json", "w") as f:
    for user_id in tqdm(starter_users):
        d = find_user_info(user_id)
        if d:
            if user_id in first_users:
                d["first"] = True
            else:
                d["first"] = False
            starter_users_info.append(d)
            f.write(json.dumps(d, ensure_ascii=False) + "\n")

100%|██████████| 88215/88215 [19:51<00:00, 74.06it/s]


In [5]:
len(starter_users_info)

81959

In [8]:
starter_users_info[0]

{'profile_image_url': 'http://pbs.twimg.com/profile_images/482744514860179456/yPRbWqz7_normal.jpeg',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'favourites_count': 4726,
 'url': None,
 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/482744514860179456/yPRbWqz7_normal.jpeg',
 'description': None,
 'id': 807990584,
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'follow_request_sent': None,
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_background_color': 'C0DEED',
 'contributors_enabled': False,
 'statuses_count': 6147,
 'profile_text_color': '333333',
 'lang': 'en',
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/807990584/1403920298',
 'verified': False,
 'default_profile_image': False,
 'created_at': 'Fri Sep 07 02:01:22 +0000 2012',
 'time_zone': None,
 'name': 'Michael Stage',
 'is_translator': False,
 'utc_offset': None,
 'notifications': None,
 'geo_enabled':

In [9]:
users_info = pd.DataFrame(starter_users_info)

In [11]:
users_info.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,first,follow_request_sent,followers_count,following,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,url,utc_offset,verified
0,False,Fri Sep 07 02:01:22 +0000 2012,True,False,None,4726,False,None,173,None,...,DDEEF6,333333,True,False,al_stage,6147,None,None,NaN,False
1,False,Wed Sep 14 02:34:41 +0000 2016,False,False,I'm a doctor. For owls. Hoot hoot!,8371,True,None,449,None,...,000000,000000,False,False,theowldoctor,7838,Pacific Time (US & Canada),http://bit.ly/2cWRQFn,-25200.0,False
2,False,Sat Jul 05 15:16:44 +0000 2014,True,False,All things considered #Liberal. #UniteBlue. I...,64,False,None,14520,None,...,DDEEF6,333333,True,False,YngLiberalGirl,13209,Athens,None,7200.0,False
3,False,Thu Mar 17 16:56:12 +0000 2016,True,True,"Trump Fan, America lover, grandfather, cab driver",24569,True,None,515,None,...,DDEEF6,333333,True,False,RBaker37115,13487,None,None,NaN,False
4,False,Mon May 17 17:27:53 +0000 2010,False,False,WEB COPY MECHANIC – Fixing Your Web Pages So T...,1,False,None,71,None,...,FFFFFF,333333,True,False,Eldo_Barkhuizen,169,London,http://arrowwrite.com,0.0,False


In [25]:
users_info.columns

Index(['contributors_enabled', 'created_at', 'default_profile',
       'default_profile_image', 'description', 'favourites_count', 'first',
       'follow_request_sent', 'followers_count', 'following', 'friends_count',
       'geo_enabled', 'id', 'id_str', 'is_translator', 'lang', 'listed_count',
       'location', 'name', 'notifications', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_banner_url', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image', 'protected',
       'screen_name', 'statuses_count', 'time_zone', 'url', 'utc_offset',
       'verified'],
      dtype='object')

In [ ]:
# user_data
def get_cdf_pdf(data, mini, maxi, step, norm=True):
    x = [float(i) for i in np.arange(mini, maxi, step)]
    cdf = [0] * len(x)
    pdf = [0] * len(x)
    count = 0
    for d in data:
        d = float(d)
        if d < mini or d > maxi:
            continue
        count += 1
        for i in range(len(x)):
            if i == (len(x) - 1) and d >= x[i]:
                pdf[i] += 1
            elif d >= x[i] and d < x[i+1]:
                pdf[i] += 1

            if d >= x[i]:
                cdf[i] += 1

    if norm:
        cdf = [c / count for c in cdf]
        pdf = [p / count for p in pdf]

    return x, cdf, pdf


plt.figure(figsize=(8, 6))
x, cdf, pdf = get_cdf_pdf(users_info['followers_count'], 0, 2000000, 500)
plt.xscale('log'); plt.yscale('log')
plt.plot(x, cdf, label='$follower\ count$')

# pd.DataFrame({'x': x, 'cdf': cdf}).to_csv('dist_follower_count.csv')

x, cdf, pdf = get_cdf_pdf(users_info['friends_count'], 0, 1000000, 500)
plt.ylabel('$CCDF$', fontsize=15)
# pd.DataFrame({'x': x, 'cdf': cdf}).to_csv('dist_following_count.csv')


plt.xticks(fontsize=14); plt.yticks(fontsize=14)
plt.xscale('log'); plt.yscale('log')

plt.plot(x, cdf, label='$following\ count$')
plt.legend(fontsize=13)
plt.savefig('fig/fol.pdf', dpi=300)
plt.show()

- 189620条谣言URL, 914272条初始推特；
- 25212个第一条谣言的用户, 88215个发初始推特的用户；

### 专门针对用户的分析，放到analyze_users.ipy中～

In [11]:
user_data = defaultdict(list)

for fake in tqdm(url_tweets):
    url = fake["url"]
    for j, t in enumerate(fake["tweets"]):
        uid = t["user_id"]
        if t["origin"]:
            if j == 0: # first
                user_data[str(uid)].append({"id": t["id"], "role": 0, "url": url, "dt": t["dt"]})
            else:
                user_data[str(uid)].append({"id": t["id"], "role": 1, "url": url, "dt": t["dt"]})
        else:
            user_data[str(uid)].append({"id": t["id"], "role": 2, "url": url, "dt": t["dt"]})
            

100%|██████████| 189620/189620 [00:02<00:00, 65628.35it/s]


In [26]:
# json.dump(tmp, open("data/user-behaviors.txt", "w"), indent=2)

user_data = json.load(open("data/user-behaviors.json"))

In [28]:
with open("data/user-behavior-features.csv", "w") as f:
    for d in user_data:
        uid = d["uid"]
        cnt_0 = 0
        cnt_1 = 0
        cnt_2 = 0
        url = set()
        for t in d["tweets"]:
            if t["role"] == 0:
                cnt_0 += 1
            elif t["role"] == 1:
                cnt_1 += 1
            elif t["role"] == 2:
                cnt_2 += 1
            url.add(t["url"])
        f.write("{},{},{},{},{},{}\n".format(uid, cnt_0, cnt_1, cnt_2, len(d["tweets"]), len(url)))
    

In [44]:
uids_of_IRA = set( [int(d) for d in json.load(open("data/IRA_match.json")).values()] )

In [31]:
user_data = pd.read_csv("data/user-behavior-features.csv")

In [48]:
cnt = 0

for i, row in user_data.iterrows():
    if row["uid"] in uids_of_IRA:
        print(row["uid"], row["cnt_url"])
        cnt += 1
        
cnt

789266125485998080 76
1679279490 61
1671234620 54
1662779209 18
4272870988 15
2606301939 15
743166519157227520 13
2491081213 13
2671070290 12
1694026190 11
2535166748 11
1661246144 10
1715424829 8
1651693646 9
1629828379 9
2572058134 7
1647045721 7
1649967228 7
4311371968 7
4224729994 6
1688370956 6
3990577513 5
2535818742 5
1833223908 5
3083844539 5
4355797094 5
4437444442 4
1533813962 4
1685718216 4
1710804738 4
1623180199 3
4289431230 3
1720117201 3
4224912857 3
4332740714 3
4208754922 3
1676481360 3
2540758755 3
4508630900 2
2570631118 2
4209684039 2
1655194147 2
1649144358 2
1690487623 2
2586506586 1
4218156466 1
1652987412 1
2533221819 1
1681162956 1
737420000043139072 1
1676377446 1
1659753702 1
3084866039 1
3312143142 1
1868981054 1
1617939218 1
1658202894 1
1653553555 1
1716228906 1


59